## Imports

In [1]:
from datasets import Dataset
from keras.utils.np_utils import to_categorical

## Constantes

In [2]:
dir = "../../"
dir_data = "data/"
dir_rotulacao = "rotulacao/result/"

arquivo_result = "facebook_200-03-02-2023.json" ## Verificar caso a caso

dir_dataset = "datasets/dataset_facebook/"## Verificar caso a caso
dataset_name = "dataset_concatenate_facebook.json" ## Verificar caso a caso

## Lendo Resultados

In [3]:
resultado_doccano = Dataset.from_json(f"{dir}{dir_data}{dir_rotulacao}{arquivo_result}")

Using custom data configuration default-a69e7c21fc397bc9
Found cached dataset json (/home/thiago/.cache/huggingface/datasets/json/default-a69e7c21fc397bc9/0.0.0)


In [4]:
resultado_doccano

Dataset({
    features: ['id', 'text', 'index', 'label', 'Comments'],
    num_rows: 200
})

## Lendo Dataset

In [5]:
dataset = Dataset.from_json(f"{dir}{dir_data}{dir_dataset}{dataset_name}")

Using custom data configuration default-919784d60fd56bf4
Found cached dataset json (/home/thiago/.cache/huggingface/datasets/json/default-919784d60fd56bf4/0.0.0)


In [6]:
dataset

Dataset({
    features: ['created_time', 'message', 'permalink_url', 'id_post', 'dataset_origem', 'index'],
    num_rows: 34620
})

## Prepara Dados Resultados

In [7]:
resultado_doccano.features

{'id': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'index': Value(dtype='int64', id=None),
 'label': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'Comments': Sequence(feature=Value(dtype='null', id=None), length=-1, id=None)}

### Remover Colunas Desnecessárias (opcional)

In [8]:
resultado_doccano = resultado_doccano.remove_columns(["id", "text", "Comments"])

### Arrumar Colunas de Label

In [9]:
def arruma_label(data):
    valor = 0
    if "Neutro" in data["label"]:
        valor = 1
    elif "Positivo" in data["label"]:
        valor = 2
    data["label"] = to_categorical(valor, num_classes=3)
    return data
resultado_doccano = resultado_doccano.map(arruma_label)

Loading cached processed dataset at /home/thiago/.cache/huggingface/datasets/json/default-a69e7c21fc397bc9/0.0.0/cache-c3022cda3fadb16c.arrow


In [10]:
resultado_doccano.features

{'index': Value(dtype='int64', id=None),
 'label': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None)}

## Prepara Dados Dataset

In [11]:
dataset.features

{'created_time': Value(dtype='string', id=None),
 'message': Value(dtype='string', id=None),
 'permalink_url': Value(dtype='string', id=None),
 'id_post': Value(dtype='string', id=None),
 'dataset_origem': Value(dtype='string', id=None),
 'index': Value(dtype='int64', id=None)}

### Adiciona a coluna de "label"

In [12]:
if "label" not in dataset.features:
    dataset = dataset.add_column("label", [None] * len(dataset))

In [13]:
dataset.features

{'created_time': Value(dtype='string', id=None),
 'message': Value(dtype='string', id=None),
 'permalink_url': Value(dtype='string', id=None),
 'id_post': Value(dtype='string', id=None),
 'dataset_origem': Value(dtype='string', id=None),
 'index': Value(dtype='int64', id=None),
 'label': Value(dtype='null', id=None)}

## Merge - Dataset & Result Doccano

### Convertendo result Doccano para Dict

In [14]:
dict_resultado_doccano = dict({dado["index"] : dado["label"] for dado in resultado_doccano})

### Merge

In [15]:
def merge(data):
    if data["index"] in dict_resultado_doccano.keys():
        data["label"] = dict_resultado_doccano[data["index"]]
    return data
dataset = dataset.map(merge)

Loading cached processed dataset at /home/thiago/.cache/huggingface/datasets/json/default-919784d60fd56bf4/0.0.0/cache-77009f12d96f26f8.arrow


## Salvando Dataset

In [16]:
dataset.to_json(f"{dir}{dir_data}{dir_dataset}{dataset_name}")

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

12978948